# ```JavaScript```, ```AJAX``` y páginas dinámicas

Hoy en día muchísimos servidores no mandan la información que tienen que mostrar las páginas directamente embebida en el código *HTML*, sino que usan *JavaScript* (y en muchas ocasiones *AJAX*) para mostrar esa información de manera dinámica. De hecho, en muchos casos esa información puede aparecer o desaparecer de la página según sea el comportamiento del usuario del navegador. Ejemplos sencillos son la página https://slashdot.org, o la página https://twitter.com/explore. Esas páginas van cargando su contenido según el usuario va haciendo _scroll_ hacia abajo.

Una página que ya conocemos a la que le pasa esto es https://covid19.isciii.es/. Si con nuestro navegador (con JavaScript activado) veremos que los datos de la tabla que parseabamos en ejemplos anteriores no son iguales a los que nos muestra el navegador. De hecho con Python solo cargabamos unos datos estáticos de un día pasado (vete tu a saber cual), mientras que con el navegador los datos de esa tabla aparecen actualizados cada día. Para poder solucionar este problema deberíamos ser capaces de interpretar el código _JavaScript_ de las páginas que escrapeamos, pero eso está totalmente fuera de la funcionalidad de _requests_, de _Beautiful Soup_, etc.

En este último _notebook_ vamos a aprender a manejar la herramienta ```Selenium```. Esta herramienta en origen estaba pensada para poder comprobar el funcionamiento de páginas web (cuando se están desarrollando) y poder implementar pruebas automáticas. Lo que hace esta herramienta es "encapsular" un navegador de nuestra elección (los más usuales están disponibles) de tal manera que podemos manejar dicho navegador programáticamente (a través de código) en vez de con movimientos de ratón, clicks y pulsaciones de teclas como estamos acostumbrados normalmente. Además _Selenium_ nos permitirá acceder al _DOM_ de los documentos que haya cargado el navegador (incluso después de que se haya ejecutado código _JavaScript_. A continuación veremos algunos ejemplos de como usar Selenium para hacer _scraping_ de este tipo de páginas.

## Instalación de ```Selenium``` y el driver de nuestro navegador

Para poder manejar ```Selenium``` y el driver para _Firefox_ debemos instalar los siguientes paquetes: ```selenium``` y ```geckodriver```. Con _conda_ lo tenemos fácil:

    > conda install -c conda-forge selenium
    > conda install -c conda-forge geckodriver
    
Si queremos instalar un driver para otro navegador distinto podemos consultar como hacerlo en la documentación u otros tutoriales:

* https://www.selenium.dev/documentation
* https://selenium-python.readthedocs.io/


In [ ]:
from selenium import webdriver
from lxml import etree
from io import StringIO, BytesIO
import matplotlib.pyplot as plt

# Utilizamos Selenium para obtener el código de la página, pero en este caso la información
# que devolverá incluirá los cambios que se hayan realizado en el DOM tras la ejecución del código
# JavaScript de la página
driver = webdriver.Firefox()
driver.get("https://zerjio.com/temp/covid19.html")

html = driver.page_source

#driver.close()

# print(html)

# A partir de aquí el código es muy muy parecido al del segundo notebook

parserHTML = etree.HTMLParser()
tree = etree.parse(StringIO(html), parserHTML)

tablas = tree.xpath('//table') 

tabla = tablas[1]

comunidades = []
casos = []

filas = tabla.xpath('tbody/tr') 

for f in filas:
    comunidades.append(f[0].text)
    casos.append(int(f[1].text))
    
plt.bar(comunidades, casos)
plt.xticks(rotation=90)
plt.show()

## Interaccionando con páginas web

Selenium nos permite interaccionar con las páginas web tal y como lo haríamos como si estuvieramos navegando manualmente con el navegador. Para ello tiene funciones que nos permiten seleccionar elementos de una página (o de otras partes del interfaz del navegador como pestañas, etc), mandar pulsaciones de teclas, clicks de ratón, etc.

In [ ]:
# Ejemplo de búsqueda en un formulario. Este ejemplo dejará de funcionar más pronto que tarde en cuanto AliExpress
# actualice su página web.

from selenium import webdriver
import time

driver = webdriver.Firefox()
driver.get("https://es.aliexpress.com/")

# Vamos a hacer una búsqueda dentro de AliExpress con el término: "mascarilla con filtro para virus"
# Podemos buscar los elementos con los que interaccionar. Las búsquedas se pueden hacer por id, name o 
# incluso XPATH: https://selenium-python.readthedocs.io/locating-elements.html

time.sleep(1)  # No hace falta, pero para ir viendo poco a poco como se interacciona

# Primero cerramos el cuadro de publicidad que nos sale
botonCerrarCuadroPublicidad = driver.find_element_by_xpath("//a[@class='close-layer']")
botonCerrarCuadroPublicidad.click()

time.sleep(1)


#Buscamos el elemento del formulario de búsqueda (en este caso un <input> con id="search-key").

cuadroTexto = driver.find_element_by_id("search-key")

cuadroTexto.clear()  # Borramos el posible contenido del cuadro de texto
time.sleep(1)
cuadroTexto.send_keys("mascarilla con filtro para virus")
time.sleep(1)

botonLupa = driver.find_element_by_xpath("//input[@class='search-button']")
botonLupa.click()


In [ ]:
# Esperar hasta que aparezca un elemento en la página (por ejemplo si sabemos que
# va a crearse con JavaScript después de un tiempo)


from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

import time

driver = webdriver.Firefox()
driver.get("https://twitter.com/explore")

# Esperamos un máximo de 10 segundos hasta que aparezca el input deseado
try:
    busquedaTwitter = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//input[@placeholder='Buscar en Twitter']"))
    )
except:
    print("Error al localizar el elemento")
    driver.quit()
    

busquedaTwitter.clear()
busquedaTwitter.send_keys("Kim Jong Un")
busquedaTwitter.send_keys(Keys.RETURN)


In [ ]:
# Mover el ratón a una posición determinada (en nuestro caso el enlace a "artículos" de Meneame)

from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains

driver = webdriver.Firefox()
driver.get("https://meneame.net")

action = ActionChains(driver)

logotipo = driver.find_element_by_xpath("//a[@href='/articles']")
action.move_to_element(logotipo).perform()


In [ ]:
# Selenium tiene muchas más posibilidades: maximizar y minimizar ventanas, jugar con las distintas 
# ventanas y pestañas, cuadros de dialogo, cookies, instalar addons, hacer capturas de pantalla, etc, etc

from selenium import webdriver 
import time

driver = webdriver.Firefox()

time.sleep(1)

for ancho in range(400, 1000, 40):
    alto = ancho * 3 / 4
    driver.set_window_size(ancho, alto)
    driver.set_window_position(1000/2 - ancho/2, 800/2 - alto/2)
    time.sleep(.1)

driver.maximize_window()
time.sleep(1)

driver.get("https://selenium-python.readthedocs.io/")

driver.save_screenshot("pantallazo.png")


## Selenium IDE

Selenium también cuenta con una extensión tanto para Firefox como para Chrome:

* https://addons.mozilla.org/es/firefox/addon/selenium-ide/
* https://chrome.google.com/webstore/detail/selenium-ide/mooikfkahbdckldjjndioackbalphokd

Dicha extensión facilita la creación de scripts basados en Selenium. Es muy recomendable que la instales y compruebes sus posibilidades.
